In [2]:
import keras
from keras.datasets import cifar10
from keras import backend as K
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model
from keras.layers import concatenate

from keras import optimizers,regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint 

In [3]:
DROPOUT=0.2 # keep 80%
CONCAT_AXIS=3
weight_decay=1e-4
DATA_FORMAT='channels_last'

In [4]:
gb_fire_no = 0
def fire_module(x,squeeze,expand,channel_axis, fire_no=None):
    global gb_fire_no
    if fire_no is None: 
        gb_fire_no += 1
        fire_no = gb_fire_no
    file_name="fire"+str(fire_no)+"_"
     
    covn1_name = file_name+"Conv1x1_1" 
    x = Conv2D(squeeze,(1,1),padding='same', name=covn1_name)(x)# valid
    x = Activation('relu')(x)

    left_name = file_name+"Conv1x1_left" 
    left = Conv2D(expand,(1,1),padding='same', name=left_name)(x)# valid
    left = Activation('relu')(left)

    right_name = file_name+"Conv3x3_right" 
    right = Conv2D(expand,(3,3),padding='same', name=right_name)(x)
    right = Activation('relu')(right)
     
    x = concatenate([left, right],axis=channel_axis)
    return x

In [6]:
def squeezenet(img_input,classes=10):
     
    x = Conv2D(96, (3, 3), strides=(2, 2), padding='same', name="conv1")(img_input)# valid 
    x = Activation('relu')(x)
     
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same',data_format=DATA_FORMAT, name="pool2")(x)
    # fire 2,3,4
    x = fire_module(x,squeeze=16,expand=64,channel_axis=CONCAT_AXIS, fire_no=1)
    x = fire_module(x,squeeze=16,expand=64,channel_axis=CONCAT_AXIS, fire_no=2)
    x = fire_module(x,squeeze=32,expand=128,channel_axis=CONCAT_AXIS, fire_no=3) 
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same',data_format=DATA_FORMAT, name="pool3")(x)
    # fire 5,6,7,8
    x = fire_module(x,squeeze=32,expand=128,channel_axis=CONCAT_AXIS, fire_no=4)
    x = fire_module(x,squeeze=48,expand=192,channel_axis=CONCAT_AXIS, fire_no=5)
    x = fire_module(x,squeeze=48,expand=192,channel_axis=CONCAT_AXIS, fire_no=6)
    x = fire_module(x,squeeze=64,expand=256,channel_axis=CONCAT_AXIS, fire_no=7)   
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2),padding='same',data_format=DATA_FORMAT, name="pool4")(x)
    # fire 9
    x = fire_module(x,squeeze=64,expand=256,channel_axis=CONCAT_AXIS, fire_no=8)
    x = Conv2D(10, (1,1),strides=(1,1), padding='same', name="covn2")(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D()(x)
    out = Dense(classes, activation='softmax')(x)
    return out 

img_input=Input(shape=(32,32,3), name="Input")
output = squeezenet(img_input)
model=Model(img_input,output)
model.summary()

# 把模型保存为图片
from keras.utils import plot_model
plot_model(model,to_file='model_png/208SqueezeNet.png', show_layer_names=True, show_shapes=True)

W0808 17:17:09.129496 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0808 17:17:09.192314 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0808 17:17:09.233818 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0808 17:17:09.238188 4321588096 deprecation_wrapper.py:119] From /Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/envpy3.7/lib/python3.7/

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 96)   2688        Input[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 96)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool2 (MaxPooling2D)            (None, 8, 8, 96)     0           activation_1[0][0]               
__________________________________________________________________________________________________
fire1_Conv